## 本周内容：HTML解析(parse HTML)及生成一连串URL爬取数据

### 爬取每一个标签下面的页面

#### 1. 拆解带有参数的URL，再从query取出参数进行拆解   
    使用urllib.parse进行解析，取出query，再将query进行拆解，取出参数成python字典
#### 2. 生成一连串新的URL进一步爬取数据
    有基地URL，加上参数字典，请求新网页连续技

In [1]:
import pandas as pd
from requests_html import HTMLSession

## Xpath解析HTML

In [2]:
# 先对单一页面进行爬取数据
url = "https://www.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get(url)
# 先取我们要找的元素的大框架，再找其后辈
main_element = r.html.xpath("//ul[@class='sojob-list']/li")

# 创建一个xpath字典，键为我们要抓的数据，值为xpath
dict_xpath={ 
    'text': {
        '学历':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_text_content(_xpath_):
    results = [i.xpath(_xpath_)[0].lxml.text_content() for i in main_element]
    return(results)

def get_text(_xpath_):
    results = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in main_element]
    return(results)

def get_href(_xpath_):
    results = [list(e.xpath(_xpath_,first = True).absolute_links)[0] \
                if len(e.xpath(_xpath_,first = True).absolute_links) >= 1 \
                else "" for e in main_element]
    return(results)

# 对主要元素进行.xpath取值
data_dict = dict()

data_dict = {k:get_text_content(v) for k,v in dict_xpath['text_content'].items()}
data_dict.update({k:get_text(v) for k,v in dict_xpath['text'].items()})
data_dict.update({k:get_href(v) for k,v in dict_xpath['href'].items()})
data = pd.DataFrame(data_dict)
data.to_excel("week3_liepin.xlsx",sheet_name="搜查结果")
data

,学历,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,统招本科,8年以上,10-15k·13薪,2020年04月18日,培训主管,无锡,大明,https://www.liepin.com/job/1927552959.shtml,https://www.liepin.com/company/3441478/
1,统招本科,10年以上,10-15k·12薪,2020年04月18日,人力资源部负责人,无锡,大明,https://www.liepin.com/job/1927552535.shtml,https://www.liepin.com/company/3441478/
2,大专及以上,3年以上,面议,2020年04月18日,医疗护理床外贸销售,嘉兴-平湖,平湖爱伦家具有限公司,https://www.liepin.com/job/1927551915.shtml,https://www.liepin.com/company/9611871/
3,统招本科,5年以上,面议,2020年04月18日,渠道高级采购经理,郑州-郑东新区,北京车集互联科技有限公司,https://www.liepin.com/job/1927551779.shtml,https://www.liepin.com/company/10199249/
4,统招本科,3年以上,8-10k·12薪,2020年04月18日,专职秘书,昆明-盘龙区,昆明英惠投资有限公司,https://www.liepin.com/job/1927550737.shtml,https://www.liepin.com/company/10244817/
5,大专及以上,5年以上,8-10k·12薪,2020年04月18日,质检部长,郑州-荥阳,河南郑矿机器有限公司,https://www.liepin.com/job/1927550319.shtml,https://www.liepin.com/company/9172950/
6,本科及以上,5年以上,面议,2020年04月18日,空调标定工程师 Climate Calibration Engineer,,浙江吉利控股集团,https://www.liepin.com/job/1927550211.shtml,https://www.liepin.com/company/911622/
7,大专及以上,2年以上,6-15k·12薪,2020年04月18日,商务运营专员,青岛-市南区,飞马滨(北京)科技研发有限公司,https://www.liepin.com/job/1927546851.shtml,https://www.liepin.com/company/12001409/
8,统招本科,3年以上,8-13k·12薪,2020年04月18日,新媒体运营/社群营销+微信推广主管,上海-杨浦区,上海复新进修学院,https://www.liepin.com/job/1927545959.shtml,https://www.liepin.com/company/12210499/
9,统招本科,5年以上,面议,2020年04月18日,高净值客户经理,上海-静安区,凯银家族办公室,https://www.liepin.com/job/1927534531.shtml,https://www.liepin.com/company/9327126/


In [20]:
## 扩张公司

主要元素 = r.html.xpath("//div[@data-selector='search-conditions']")
print(主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print(x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print(x)
    
公司数据选择器链结 = r.html.xpath("//div[@data-selector='search-conditions']")[0] \
                        .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                        .xpath('//div[contains(@class,"hot-comp-tags")]/a/@href')
公司数据选择器链结

# 需要知道个对应到什么数据
公司数据选择器链结 = r.html.xpath("//div[@data-selector='search-conditions']")[0] \
                        .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                        .xpath('//div[contains(@class,"hot-comp-tags")]/a')
公司数据选择器链结

公司数据选择器 = {  x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 公司数据选择器链结}
公司数据选择器

[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'中国500强': '/zhaopin/?init=-1&headckid=fd822234e05a8e6e&flushckid=1&fromSearchBtn=2&compTag=155&ckid=6bb508b27eb78b71&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=295ebedf8ddf1bb2671d0617326dc6c6&d_curPage=0&d_pageSize=40&d_headId=295ebedf8ddf1bb2671d0617326dc6c6',
 '2018互联网300强': '/zhaopin/?init=-1&headckid=fd822234e05a8e6e&flushckid=1&fromSearchBtn=2&compTag=182&ckid=6bb508b27eb78b71&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=295ebedf8ddf1bb2671d0617326dc6c6&d_curPage=0&d_pageSize=40&d_headId=295ebedf8ddf1bb2671d0617326dc6c6',
 '制造业500强': '/zhaopin/?init=-1&headckid=fd822234e05a8e6e&flushckid=1&fromSearchBtn=2&compTag=186&ckid=6bb508b27eb78b71&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=295ebedf8ddf1bb2671d0617326dc6c6&d_curPage=0&d_

## urllib3解析url
* 为什么要解析url?   
   因为上面字典的url太长太乱，我们很难用肉眼分辨出他们的不同之处，所以需要解析他、拆分他进行比较。
* urlparse解析后返回6个部分  
   scheme（机制）、netioc（网络位置）、path（路径）、params（路径段参数）、query（查询）、fragment（片段）
* parse_qs解析query   
* nuninque进行计量对比

In [4]:
# 解析url,使用 urllib.parse 解析
from urllib.parse import urlparse,parse_qs
[ urlparse(x) for x in 公司数据选择器.values() ]

[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=fd822234e05a8e6e&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=fd822234e05a8e6e&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=295ebedf8ddf1bb2671d0617326dc6c6&d_curPage=0&d_pageSize=40&d_headId=295ebedf8ddf1bb2671d0617326dc6c6', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=fd822234e05a8e6e&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=fd822234e05a8e6e&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=295ebedf8ddf1bb2671d0617326dc6c6&d_curPage=0&d_pageSize=40&d_headId=295ebedf8ddf1bb2671d0617326dc6c6', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=fd822234e05a8e6e&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=fd822234e05a8e6e&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom

In [5]:
# 先将上面解析url的结果转换成dataframe
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器.values() ])

# 使用过nuninque()相异值计量对比
# 显示query有不同
print(df.nunique())

df.head()

scheme      1
netloc      1
path        1
params      1
query       6
fragment    1
dtype: int64


,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=fd822234e05a8e6e&flushckid=1&...,
1,,,/zhaopin/,,init=-1&headckid=fd822234e05a8e6e&flushckid=1&...,
2,,,/zhaopin/,,init=-1&headckid=fd822234e05a8e6e&flushckid=1&...,
3,,,/zhaopin/,,init=-1&headckid=fd822234e05a8e6e&flushckid=1&...,
4,,,/zhaopin/,,init=-1&headckid=fd822234e05a8e6e&flushckid=1&...,


In [6]:
# 针对  query 再解析 ，parse_qs 会返回 query（查询）多个部分
# parse_qs 对 query 进行拆分，拆分依据是符号 &
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df["query"]])
# 显示camptag有不同
print(df_qs.nunique())

df_qs.head()
df_qs[["compTag","keyword"]]
# comptag 是不同的公司选择器，keyword 是搜查关键字

ckid             1
compTag          6
d_ckId           1
d_curPage        1
d_headId         1
d_pageSize       1
d_sfrom          1
flushckid        1
fromSearchBtn    1
headckid         1
init             1
keyword          1
siTag            1
dtype: int64


,compTag,keyword
0,155,PRD
1,182,PRD
2,186,PRD
3,189,PRD
4,130,PRD
5,156,PRD


In [11]:
# 对上一段代码进行说明
# df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df["query"]])
# 等于下面这一段代码

#   list_1 = []
#   for q in df["query"]:
#     dict_query = {}
#     for k,v in parse_qs(q).items():
#         dict.update({k:v[0]})
#     list_1.append(dict_query)
#   print(dict_query)
#   print(list_1)


In [12]:
## 构建参数模板 及 字典_comptag
def parse_url_qs_for_compTag(url):
    six_parts = urlparse(url)
    out = parse_qs(six_parts.query)
    return out

参数模板 = parse_url_qs_for_compTag(list(公司数据选择器.values())[0])
print(参数模板)

dict_compTag = {k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器.items()}
print(dict_compTag)

{'init': ['-1'], 'headckid': ['fd822234e05a8e6e'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['PRD'], 'compTag': ['155'], 'ckid': ['fd822234e05a8e6e'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['295ebedf8ddf1bb2671d0617326dc6c6'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['295ebedf8ddf1bb2671d0617326dc6c6']}
{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


In [13]:
# 构建参数模板
def 参数模板生成(compTag,keyword):
    参数 = 参数模板.copy()
    参数['compTag'] = compTag
    参数['keyword'] = keyword
    return(参数)

参数_compTag_用户体验 = {k:参数模板生成(compTag = [v],keyword = ["用户体验"]) for k,v in dict_compTag.items()}
print(参数_compTag_用户体验)

{'中国500强': {'init': ['-1'], 'headckid': ['fd822234e05a8e6e'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['155'], 'ckid': ['fd822234e05a8e6e'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['295ebedf8ddf1bb2671d0617326dc6c6'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['295ebedf8ddf1bb2671d0617326dc6c6']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['fd822234e05a8e6e'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['182'], 'ckid': ['fd822234e05a8e6e'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['295ebedf8ddf1bb2671d0617326dc6c6'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['295ebedf8ddf1bb2671d0617326dc6c6']}, '制造业500强': {'init': ['-1'], 'headckid': ['fd822234e05a8e6e'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['186'], 'ckid': ['fd822234e05a8e6e'], '

## requests生成

In [14]:
# 准备测试多个页面
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验["中国500强"]
r = session.get(url,params = payload)
r.url

'https://www.liepin.com/zhaopin/?init=-1&headckid=fd822234e05a8e6e&flushckid=1&fromSearchBtn=2&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&compTag=155&ckid=fd822234e05a8e6e&siTag=1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=295ebedf8ddf1bb2671d0617326dc6c6&d_curPage=0&d_pageSize=40&d_headId=295ebedf8ddf1bb2671d0617326dc6c6'

In [17]:
# 先爬取单个页面
session = HTMLSession()

def requests_liepin( url,params):
    r = session.get(url,params = payload)
    _main = r.html.xpath("//ul[@class='sojob-list']/li")
    dict_xpath = {
        "text":{
            "学历": '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            "经验": '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            "薪水": '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            "时间": '//div[contains(@class,"job-info")]/p/time/@title', 
            "职称": '//div[contains(@class,"job-info")]/h3/a', 
            "公司地点": '//div[contains(@class,"job-info")]/p/a',
            "公司名称": '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a',            
        },
        "text_content":{
        },
        "href":{
            "链接": '//div[contains(@class,"job-info")]/h3/a',
            "公司url": '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a',
        }
    }
    
    def get_text_content(_xpath_):
        result = [e.xpath(_xpath_)[0].lxml.text_content() for e in _main]
        return(result)
    def get_text(_xpath_):
        result = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in _main]
        return(result)
    def get_e_href(_xpath_):
        result = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in _main]
        return(result)
    
    数据字典 = dict()

    数据字典 = {k:get_text_content(v) for k,v in dict_xpath['text_content'].items()}
    数据字典.update({k:get_text(v) for k,v in dict_xpath['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpath['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)


In [18]:
# 多个页面爬取
url = "https://www.liepin.com/zhaopin/"
list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin(url,params = payload)   # requests_liepin是对整个页面进行xpath
    df = df.assign(热门公司类型 = k)
    list_df.append(df)
    
df_all =pd.concat(list_df)  # 合并数据
df_all

,学历,经验,薪水,时间,职称,公司地点,公司名称,链接,公司url,热门公司类型
0,本科及以上,5年以上,面议,2020年04月18日,资深栏目运营,杭州,网易集团,https://www.liepin.com/job/1927388909.shtml,https://www.liepin.com/company/5964833/,中国500强
1,本科及以上,1年以上,20-40k·12薪,2020年04月18日,高级客户经理/客户经理,深圳-龙岗区,阿里巴巴,https://www.liepin.com/job/1927547453.shtml,https://www.liepin.com/company/8679040/,中国500强
2,本科及以上,3年以上,30-50k·16薪,2020年04月16日,数据技术及产品部-数据研发专家-数据资产平台,杭州-余杭区,阿里巴巴,https://www.liepin.com/job/1927492123.shtml,https://www.liepin.com/company/1072424/,中国500强
3,统招本科,经验不限,面议,2020年04月16日,达标车型认证岗,济南-高新区,中国重型汽车集团有限公司,https://www.liepin.com/job/1927181655.shtml,https://www.liepin.com/company/9739005/,中国500强
4,本科及以上,5年以上,20-35k·12薪,2020年04月15日,税务筹划专家,郑州,郑州宇通集团有限公司,https://www.liepin.com/job/1927303523.shtml,https://www.liepin.com/company/1530833/,中国500强
5,统招本科,5年以上,30-50k·12薪,2020年04月13日,集团总部投资总监,深圳-南山区,禹洲地产股份有限公司,https://www.liepin.com/job/1927409715.shtml,https://www.liepin.com/company/2245242/,中国500强
6,硕士及以上,10年以上,25-50k·12薪,2020年04月13日,课程研发负责人,上海-罗店,中国金茂控股集团有限公司,https://www.liepin.com/job/1926937565.shtml,https://www.liepin.com/company/6990643/,中国500强
7,硕士及以上,10年以上,85-115k·12薪,2020年04月13日,执行校长,上海-宝山区,中国金茂控股集团有限公司,https://www.liepin.com/job/1926933163.shtml,https://www.liepin.com/company/6990643/,中国500强
8,本科及以上,1年以上,面议,2020年04月03日,海外商务经理,,网易集团,https://www.liepin.com/job/1924796421.shtml,https://www.liepin.com/company/5964833/,中国500强
9,大专及以上,2年以上,面议,2020年04月03日,广告销售大客户经理,,网易集团,https://www.liepin.com/job/1919211941.shtml,https://www.liepin.com/company/5964833/,中国500强


In [19]:
# 输出多页搜查结果
df_all.to_excel("web_mining_week3.xlsx",sheet_name="搜查结果")